In [ ]:
# Calculations
import numpy as np
import xarray as xr
import pandas as pd
import os

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import cmocean as cmo

In [ ]:
# Specify path experiment
exp_class = 'CD-Paolo'
run = 'r001'
experiment_path = f'../experiments/{exp_class}/{run}/AA_{run}.nc'

# Create directory
dirName = f'animations/{exp_class}'
try:
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ")
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

save_name = f'animations/{exp_class}/{run}'
print(f'Save directory of animations is: {save_name}')


In [ ]:
# mask data that falls out of shelf mask
def mask_data(ds):
    for vars in ['S','T','D','melt','entr','ent2','detr']:
        ds[vars] = xr.where(ds.tmask==1,ds[vars],np.nan)
    ds['U'] = xr.where(ds.umask==1,ds['U'],np.nan)
    ds['V'] = xr.where(ds.vmask==1,ds['V'],np.nan)
    return ds

# Save mean values of U, V, D, T, S, melt over the ice shelf -> time-series of the mean variables (need umask/vmask)
def compute_timeseries_mean(ds_masked, vars):
    
    ds_means=[]

    for var in vars:
        ds_meanvar = ds_masked[var]
        ds_meanvar = ds_meanvar.mean(dim=['x','y'])#.rename(f'mean_{var}')
        ds_means.append(ds_meanvar)

    ds_mean = xr.merge(ds_means)
    #ds_mean = ds_mean.assign(update_frequency = ds_masked.update_frequency)
    #ds_mean = ds_mean.assign(start_year = ds_masked.start_year)
    #ds_mean = ds_mean.assign(end_year = ds_masked.end_year)

    return ds_mean

In [ ]:
# Read data, mask data out of ice shelf
ds = xr.open_dataset(f'{experiment_path}')
ds_m = mask_data(ds)

mean_variables = ['melt','D','T','S','U','V','entr','ent2','detr']
mean_ds = compute_timeseries_mean(ds_m, mean_variables)


In [ ]:
# Define dictionaries for vmin, vmax, cmap en labels associated with the different model variables

# OC3
#vmins = {'melt':-50, 'D':0,   'T':-2.0, 'S':33.7, 'U':-0.4, 'V':-0.4,'entr': 0, 'ent2': 0, 'detr':0, 'draft':-400}
#vmaxs = {'melt':50,  'D':200, 'T':-1.0, 'S':34.3, 'U':0.4,  'V':0.4,'entr': 10, 'ent2': 10, 'detr':10,'draft': 0}

# OC4
vmins = {'melt':-5, 'D':0,   'T':-2.5, 'S':33.7, 'U':-0.4, 'V':-0.4, 'entr': 1, 'ent2': 1, 'detr':0, 'draft':-400,'mask':0,'tmask':0}
vmaxs = {'melt':5,  'D':200, 'T':-1.5, 'S':34.3, 'U':0.4,  'V':0.4,'entr': 2000, 'ent2': 2000, 'detr':10, 'draft':0,'mask':3,'tmask':1}

cmaps = {'melt':cmo.cm.balance, 'D':cmo.cm.dense, 'T':cmo.cm.thermal, 'S':cmo.cm.haline, 'U':cmo.cm.balance, 'V':cmo.cm.balance, 'entr': 'Oranges', 'ent2': 'Oranges', 'detr':'Blues','mask':cmo.cm.haline,'draft':cmo.cm.deep,'tmask':cmo.cm.haline}
labs =  {'melt':'Melt [m/yr]',  'D':'D [m]',      'T':'T [degC]',     'S':'S [psu]',     'U':'U [m/s]',      'V':'V [m/s]', 'entr': 'entr', 'ent2':'ent2', 'detr':'detr','mask':'mask', 'draft':'draft','tmask':'tmask'}


# REALISTIC
vmins = {'melt':1, 'D':0,   'T':-1.9, 'S':33.7, 'U':-0.4, 'V':-0.4, 'entr': 1, 'ent2': 1, 'detr':0, 'draft':-1000,'mask':0,'tmask':0}
vmaxs = {'melt':100,  'D':200, 'T':2.5, 'S':34.3, 'U':0.4,  'V':0.4,'entr': 2000, 'ent2': 2000, 'detr':10, 'draft':0,'mask':3,'tmask':1}
cmaps = {'melt':'inferno', 'D':cmo.cm.dense, 'T':cmo.cm.thermal, 'S':cmo.cm.haline, 'U':cmo.cm.balance, 'V':cmo.cm.balance, 'entr': 'Oranges', 'ent2': 'Oranges', 'detr':'Blues','mask':cmo.cm.haline,'draft':cmo.cm.deep,'tmask':cmo.cm.haline}
labs =  {'melt':'Melt [m/yr]',  'D':'D [m]',      'T':'T [degC]',     'S':'S [psu]',     'U':'U [m/s]',      'V':'V [m/s]', 'entr': 'entr', 'ent2':'ent2', 'detr':'detr','mask':'mask', 'draft':'draft','tmask':'tmask'}

#vmins = {'melt':-10, 'D':0,   'T':-1.9, 'S':33.7, 'U':-0.4, 'V':-0.4, 'entr': 1, 'ent2': 1, 'detr':0, 'draft':-1000,'mask':0,'tmask':0}
#vmaxs = {'melt':10,  'D':200, 'T':2.5, 'S':34.3, 'U':0.4,  'V':0.4,'entr': 2000, 'ent2': 2000, 'detr':10, 'draft':0,'mask':3,'tmask':1}

#cmaps = {'melt':cmo.cm.balance, 'D':cmo.cm.dense, 'T':cmo.cm.thermal, 'S':cmo.cm.haline, 'U':cmo.cm.balance, 'V':cmo.cm.balance, 'entr': 'Oranges', 'ent2': 'Oranges', 'detr':'Blues','mask':cmo.cm.haline,'draft':cmo.cm.deep,'tmask':cmo.cm.haline}
#labs =  {'melt':'Melt [m/yr]',  'D':'D [m]',      'T':'T [degC]',     'S':'S [psu]',     'U':'U [m/s]',      'V':'V [m/s]', 'entr': 'entr', 'ent2':'ent2', 'detr':'detr','mask':'mask', 'draft':'draft','tmask':'tmask'}





In [ ]:
def compute_snapshots(dataset, time_range, variable=None):
    
    snapshots=[]

    for i in time_range:
        if variable == None:
            snapshots.append(dataset.sel(t=i))
        else:
            snapshots.append(dataset.sel(t=i)[variable])

    return snapshots


def animation_vars(dataset, variables, time_range, fps, meanvars=None, savename=None):
    """
    This function generates and possibly saves animations of 2D variables from dataset ds.
    
    Input:
        - ds: xarray dataset, with dimensions [x,y,t]
        - vars: 2D variables to be plotted, e.g. ['melt','T']
        - time_range: array of days, e.g. np.arange(1,20,1)
    
     """

    nVars = len(variables)

    snapshots = [None] * nVars
    ims = [None] * nVars
    ims_g = [None] * nVars
    ims_o = [None] * nVars

    #UF = dataset.update_frequency.values
    UF = 1
    if nVars > 2:
        fig, ax = plt.subplots(2, 3, figsize = (6*3,2*4.5))
    elif nVars == 2:
        fig, ax = plt.subplots(nVars,1,figsize=(6,nVars*4.5))

    ax = ax.flatten()
    #compute snapshots and generate initial figure
    ocean_mask  = xr.where(dataset.mask==0, 1, np.nan)
    ground_mask = xr.where(dataset.mask==2, 1, np.nan)

    ocean_ss = compute_snapshots(ocean_mask, time_range)
    ground_ss = compute_snapshots(ground_mask, time_range)

    meanmelt = []
    for i in time_range:
        meanmelt.append(meanvars.sel(t=i).melt.values)

    for j, var in enumerate(variables):
        snapshots[j] = compute_snapshots(dataset, time_range, var)
        ini = snapshots[j][0]

        if var == 'entr' or var == 'ent2' or var == 'melt':
            normentr=mpl.colors.LogNorm(vmin=vmins[var],vmax=vmaxs[var])
            ims[j] = ax[j].pcolormesh(ini.x, ini.y, ini, norm=normentr, cmap=cmaps[var])
        else:
            ims[j] = ax[j].pcolormesh(ini.x, ini.y, ini, vmin=vmins[var], vmax=vmaxs[var], cmap=cmaps[var])

        #ims[j] = ax[j].pcolormesh(ini.x, ini.y, ini, vmin=vmins[var], vmax=vmaxs[var], cmap=cmaps[var])
        cbar = fig.colorbar(ims[j], ax = ax[j])
        cbar.set_label(labs[var])

        ims_o[j] = ax[j].pcolormesh(ini.x,ini.y,ocean_mask.sel(t=1),vmin=0,vmax=3,cmap='gray')
        ims_g[j] = ax[j].pcolormesh(ini.x,ini.y,ground_mask.sel(t=1),vmin=0,vmax=1.5,cmap='gray')

        ax[j].set_title(f'{var}: Day = {snapshots[j][0].t.values}, Year = {np.floor(snapshots[j][0].t.values/UF)}')

        if var == 'melt':
            x = dataset.x.values[0]
            y = dataset.y.values[0]
            text = ax[j].text(x,y,f'{meanmelt[0]:.2f} m/yr',ha='left')

    def animate_func(i):
        if i % fps == 0:
            print( '.', end ='' )
        for j, var in enumerate(variables):
            ims_g[j].set_array(ground_ss[i])
            ims_o[j].set_array(ocean_ss[i])
            
            ims[j].set_array(snapshots[j][i])

            ax[j].set_title(f'{var}: Day = {snapshots[j][i].t.values}, Year = {np.floor((snapshots[j][i].t.values-1)/UF)}')

            if var == 'melt':
                text.set_text(f'{meanmelt[i]:.2f} m/s')

        return [ims]

    anim = animation.FuncAnimation(
                               fig, 
                               animate_func, 
                               frames = len(time_range),
                               interval = 1000 / 400, # in ms
                               )

    fig.tight_layout()
    
    anim.save(f'{savename}.mp4', fps=fps, extra_args=['-vcodec', 'libx264'])

    print('Done!')


In [ ]:
#ds_slice = ds_m.sel(x=slice(400000,650000))
animation_vars(ds_m, ['melt','draft'], np.arange(1,21,1), meanvars=mean_ds, fps=10, savename=f'{save_name}')